In [1]:
import random
import os
os.environ["HF_HOME"] = r"./.cache"

from transformers import EncoderDecoderModel, AutoTokenizer, GenerationConfig, Seq2SeqTrainer, Seq2SeqTrainingArguments
from tokenizers import processors
import evaluate

- Encoders
    - BERT_JA : `cl-tohoku/bert-base-japanese-v3`
    - BERT_EN : `bert-base-uncased`, `prajjwal1/bert-tiny`
- Decorders
    - GPT_JA : `rinna/japanese-gpt2-xsmall`
    - GPT_EN : `gpt2`

In [2]:
source_lng = "ja"

if source_lng == "en":
    target_lng = "ja"
    encoder = "bert-base-uncased"
    decoder = "rinna/japanese-gpt2-small"
else: 
    target_lng = "en"
    encoder = "cl-tohoku/bert-base-japanese-v3"
    decoder = "gpt2"

model = EncoderDecoderModel.from_encoder_decoder_pretrained(
    encoder, decoder, encoder_add_pooling_layer=False
)
model.cuda();

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.7.crossattention.c_attn.bias', 'h.7.crossattention.c_proj.weight', 'h.6.crossattention.c_proj.bias', 'h.0.crossattention.c_proj.bias', 'h.10.crossattention.c_proj.bias', 'h.7.crossattention.q_attn.weight', 'h.0.crossattention.c_attn.bias', 'h.1.crossattention.c_attn.weight', 'h.8.ln_cross_attn.weight', 'h.11.crossattention.c_attn.weight', 'h.1.ln_cross_attn.weight', 'h.5.crossattention.c_proj.weight', 'h.4.crossattention.c_attn.weight', 'h.3.crossattention.q_attn.weight', 'h.5.crossattention.c_attn.bias', 'h.6.crossattention.q_attn.weight', 'h.1.crossattention.q_attn.bias', 'h.7.crossattention.c_proj.bias', 'h.4.ln_cross_attn.bias', 'h.6.crossattention.c_attn.bias', 'h.9.crossattention.c_attn.weight', 'h.2.ln_cross_attn.bias', 'h.0.crossattention.c_attn.weight', 'h.8.crossattention.c_proj.bias', 'h.9.crossattention.c_attn.bias', 'h.8.crossattention.q_attn.weight', 'h.1.

In [3]:
def print_model_parameters():
    t_pars, t_bytes = 0, 0
    for p in model.parameters():
        t_pars += p.nelement()
        t_bytes += p.nelement() * p.element_size()

    c_attn_pars, c_attn_bytes = 0, 0
    for layer in model.decoder.transformer.h:
        for p in layer.crossattention.parameters():
            c_attn_pars += p.nelement()
            c_attn_bytes += p.nelement() * p.element_size()
        for p in layer.ln_cross_attn.parameters():
            c_attn_pars += p.nelement()
            c_attn_bytes += p.nelement() * p.element_size()

    print(f"Total number of parameters: {t_pars:12,} ({(t_bytes / 1024**2):7,.1f}MB)")
    print(f"Cross-attention parameters: {c_attn_pars:12,} ({(c_attn_bytes / 1024**2):7,.1f}MB)")

print_model_parameters()

Total number of parameters:  263,423,232 (1,004.9MB)
Cross-attention parameters:   28,366,848 (  108.2MB)


In [4]:
def set_cross_attention_only(model):
    for p in model.parameters():
        p.requires_grad = False
    for layer in model.decoder.transformer.h:
        for p in layer.crossattention.parameters():
            p.requires_grad = True
        for p in layer.ln_cross_attn.parameters():
            p.requires_grad = True
set_cross_attention_only(model)

In [5]:
encoder_tokenizer = AutoTokenizer.from_pretrained(encoder, use_fast=True)
decoder_tokenizer = AutoTokenizer.from_pretrained(decoder, use_fast=True)
if decoder_tokenizer.pad_token_id is None:
    decoder_tokenizer.pad_token_id = decoder_tokenizer.eos_token_id

model.config.decoder_start_token_id = decoder_tokenizer.bos_token_id
model.config.eos_token_id = decoder_tokenizer.eos_token_id
model.config.pad_token_id = decoder_tokenizer.eos_token_id

# add EOS token at the end of each sentence
decoder_tokenizer._tokenizer.post_processor = processors.TemplateProcessing(
    single="$A " + decoder_tokenizer.eos_token,
    special_tokens=[(decoder_tokenizer.eos_token, decoder_tokenizer.eos_token_id)],
)

In [6]:
from utils.dataset import EnJaDatasetMaker
from transformers import DataCollatorForSeq2Seq

dataset = EnJaDatasetMaker.load_dataset("ja-en-test-1")
train_data = dataset.select(range(100))
valid_data = dataset.select(range(100, 150))

data_collator = DataCollatorForSeq2Seq(encoder_tokenizer, model=model)

In [7]:
metric = evaluate.load("sacrebleu")

def compute_metrics(preds):
    preds_ids, labels_ids = preds

    labels_ids[labels_ids == -100] = decoder_tokenizer.eos_token_id
    references = decoder_tokenizer.batch_decode(labels_ids, skip_special_tokens=True)
    references = [[reference] for reference in references]

    predictions = decoder_tokenizer.batch_decode(preds_ids, skip_special_tokens=True)

    if target_lng == "ja":
        bleu_output = metric.compute(
            references=references, 
            predictions=predictions, 
            tokenize="ja-mecab"
        )
    else:
        bleu_output = metric.compute(
            references=references, 
            predictions=predictions
        )
    return bleu_output


In [8]:
MAX_LENGHT = 128
def set_decoder_configuration(gc: GenerationConfig):
    gc.no_repeat_ngram_size = 3
    gc.length_penalty = 2.0
    gc.num_beams = 3
    #gen_config.max_new_tokens = MAX_LENGHT
    gc.max_length = MAX_LENGHT * 2
    gc.min_length = 0
    gc.early_stopping = True
    gc.pad_token_id = decoder_tokenizer.eos_token_id
    gc.bos_token_id = decoder_tokenizer.bos_token_id
    gc.eos_token_id = decoder_tokenizer.eos_token_id
    return gc

gen_config = GenerationConfig()
gen_config = set_decoder_configuration(gen_config)

In [9]:
train_args = Seq2SeqTrainingArguments(
    report_to="wandb",
    run_name="testing-data-maker-2",
    num_train_epochs=50,

    logging_strategy="steps",
    logging_steps=10,

    evaluation_strategy="epoch",

    output_dir="./.ckp/",
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=4,

    optim="adamw_torch",
    bf16=True,

    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    
    group_by_length=True,
    length_column_name="length",

    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_config=gen_config,
    # torch_compile=True,
    # label_smoothing_factor=0,
    # auto_find_batch_size=False,
)

In [10]:
trainer = Seq2SeqTrainer(
    model, 
    args=train_args,
    data_collator=data_collator,
    train_dataset=train_data, 
    eval_dataset=valid_data, 
    compute_metrics=compute_metrics
)

In [11]:
model.train()
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: davidboening (dandd). Use `wandb login --relogin` to force relogin


  0%|          | 0/650 [00:00<?, ?it/s]

c:\src\nlp-project\venv\lib\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


{'loss': 5.0748, 'learning_rate': 4.923076923076924e-05, 'epoch': 0.77}


  0%|          | 0/7 [00:00<?, ?it/s]

...

  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 5.447695732116699, 'eval_score': 1.0001238489314928, 'eval_counts': [74, 5, 1, 0], 'eval_totals': [448, 398, 348, 298], 'eval_precisions': [16.517857142857142, 1.256281407035176, 0.28735632183908044, 0.16778523489932887], 'eval_bp': 1.0, 'eval_sys_len': 448, 'eval_ref_len': 419, 'eval_runtime': 3.656, 'eval_samples_per_second': 13.676, 'eval_steps_per_second': 1.915, 'epoch': 49.0}
{'loss': 1.3264, 'learning_rate': 7.692307692307694e-07, 'epoch': 49.23}
{'loss': 1.2709, 'learning_rate': 0.0, 'epoch': 50.0}


  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 5.450464248657227, 'eval_score': 0.5844467086477805, 'eval_counts': [75, 3, 0, 0], 'eval_totals': [473, 423, 373, 323], 'eval_precisions': [15.856236786469344, 0.7092198581560284, 0.13404825737265416, 0.07739938080495357], 'eval_bp': 1.0, 'eval_sys_len': 473, 'eval_ref_len': 419, 'eval_runtime': 3.808, 'eval_samples_per_second': 13.13, 'eval_steps_per_second': 1.838, 'epoch': 50.0}
{'train_runtime': 261.0817, 'train_samples_per_second': 19.151, 'train_steps_per_second': 2.49, 'train_loss': 1.9977916130652795, 'epoch': 50.0}


TrainOutput(global_step=650, training_loss=1.9977916130652795, metrics={'train_runtime': 261.0817, 'train_samples_per_second': 19.151, 'train_steps_per_second': 2.49, 'train_loss': 1.9977916130652795, 'epoch': 50.0})

In [12]:
model.cuda()
model.eval()
train_out = trainer.predict(train_data)
valid_out = trainer.predict(valid_data)

print("Train:", compute_metrics((train_out.predictions, train_data["labels"])))
print("Valid:", compute_metrics((valid_out.predictions, valid_data["labels"])))

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

Train: {'score': 9.366630959325185, 'counts': [215, 67, 36, 24], 'totals': [794, 694, 594, 494], 'precisions': [27.07808564231738, 9.654178674351584, 6.0606060606060606, 4.8582995951417], 'bp': 1.0, 'sys_len': 794, 'ref_len': 740}
Valid: {'score': 0.5805108167112687, 'counts': [73, 3, 0, 0], 'totals': [473, 423, 373, 323], 'precisions': [15.433403805496829, 0.7092198581560284, 0.13404825737265416, 0.07739938080495357], 'bp': 1.0, 'sys_len': 473, 'ref_len': 409}


In [13]:
train_decode = decoder_tokenizer.batch_decode(train_out.predictions, skip_special_tokens=True)
valid_decode = decoder_tokenizer.batch_decode(valid_out.predictions, skip_special_tokens=True)

In [14]:
def print_pairs(dataset, generation, sample=5):
    assert len(dataset) == len(generation), "Invalid combination!"

    sample_ids = random.sample(range(len(dataset)), sample)
    for i, sid in enumerate(sample_ids):
        print(f"Sentence #{i} [id={sid}]")
        print(
            f"\tOriginal:  {dataset['source'][sid]}\n"
            f"\tTarget:    {dataset['target'][sid]}\n"
            f"\tGenerated: {generation[sid]}\n"
        )
    return

print_pairs(train_data, train_decode, sample=3)

Sentence #0 [id=81]
	Original:  花を入れるものには何本の花が入っていますか。
	Target:    how many flowers are there in the vase ?
	Generated: the book is full of historical facts.

Sentence #1 [id=14]
	Original:  私たちはローマで楽しく過ごしてます。
	Target:    we are having a nice time in rome .
	Generated: i like coffee.

Sentence #2 [id=3]
	Original:  彼女はとても美しい。その上、とても賢い。
	Target:    she is very beautiful , and what is more , very wise .
	Generated: i am very young.

